## Data download

In [3]:
from transform import download_transform_load
from const import *
tickers = ["jsw", "lwb", "kgh", "cla"]
for ticker in tickers:
    download_transform_load(ticker, DATA_DOWNLOAD_SOURCE.STOOQ_PL, DATABASE_TYPE.SQLITE)

INFO:src.preprocessing.DataDownloader:Stock returned jsw
INFO:src.preprocessing.DataTransformer:Dataframe cleaned
2025-12-15 06:43:15,740 [INFO] root StorageSQLite.py: Dataframe saved to database
2025-12-15 06:43:16,934 [INFO] root StorageSQLite.py: Dataframe saved to database
2025-12-15 06:43:18,193 [INFO] root StorageSQLite.py: Dataframe saved to database
2025-12-15 06:43:19,219 [INFO] root StorageSQLite.py: Dataframe saved to database


## get dataframe and some features

In [21]:
from src.eda.FeatureEngineer import FeatureEngineer
from src.storage.storage_factory import get_storage
from const import *
storage_inst = get_storage(DATABASE_TYPE.SQLITE)
df_jsw = storage_inst.data_load("jsw")
jsw_featureEngineer = FeatureEngineer(df_jsw)
jsw_featureEngineer.add_returns()
jsw_featureEngineer.add_Simple_Moving_Average(period=14)
jsw_featureEngineer.add_RSI()
jsw_featureEngineer.handle_nan_values()
df_jsw = df_jsw.dropna()
print(df_jsw.columns)

2025-12-15 13:20:31,918 [INFO] root StorageSQLite.py: Dataframe loaded from database


Index(['open', 'high', 'low', 'close', 'volume', 'returns', 'SMA', 'RSI'], dtype='object')


In [22]:
from sklearn.model_selection import train_test_split
df_jsw = df_jsw.reset_index().drop("date", axis=1)
period = 30
x_data = df_jsw.iloc[period:]
y_data = df_jsw["close"].shift(period).dropna()
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, shuffle=False)

In [23]:
from src.forcasting.pipeline import get_Linear_Regression_pipeline
Preprocessor, Model = get_Linear_Regression_pipeline()
preprocessor = Preprocessor()
model = Model()

In [24]:
preprocessor.fit(x_train)
x_train = preprocessor.transform(x_train)
x_test = preprocessor.transform(x_test)

In [25]:
model.train_model((x_train, y_train))

In [26]:
model.test_model((x_test, y_test))

0.8722402575077209
